# Hyperparameter Tuning using HyperDrive

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [2]:
from azureml.core import Workspace, Experiment
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice
from azureml.core import ScriptRunConfig 
import os



## Dataset

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [3]:
ws = Workspace.from_config()
experiment_name = 'breast-cancer-hyperdrive'

experiment=Experiment(ws, experiment_name)

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')



Workspace name: quick-starts-ws-251697
Azure region: westus2
Subscription id: 9b72f9e6-56c5-4c16-991b-19c652994860
Resource group: aml-quickstarts-251697


In [4]:
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException

# Choose a name for your CPU cluster
amlcompute_cluster_name = "azurecapstone-project"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',# for GPU, use "STANDARD_NC6"
                                                           #vm_priority = 'lowpriority', # optional
                                                           max_nodes=4)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

    compute_target.wait_for_completion(show_output=True, min_node_count = 1, timeout_in_minutes = 5)




Found existing cluster, use it.


In [6]:
from azureml.core.dataset import Dataset
from sklearn.model_selection import train_test_split

key = 'BreastCancerDataset'
if key in ws.datasets.keys():
    dataset = ws.datasets[key]
    print("Dataset found!")

else:
    print("Register dataset")

Dataset found!


In [7]:
df = dataset.to_pandas_dataframe()
train, test = train_test_split(df, shuffle=True)
train.to_csv('train.csv',index = False)

datastore = ws.get_default_datastore()
datastore.upload_files(files = ['./train.csv'])

train = Dataset.Tabular.from_delimited_files([(datastore,'train.csv')])



"datastore.upload_files" is deprecated after version 1.0.69. Please use "FileDatasetFactory.upload_directory" instead. See Dataset API change notice at https://aka.ms/dataset-deprecation.


Uploading an estimated of 1 files
Uploading ./train.csv
Uploaded ./train.csv, 1 files out of an estimated total of 1
Uploaded 1 files


## Hyperdrive Configuration

TODO: Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.

In [11]:
from azureml.core import Environment, ScriptRunConfig

early_termination_policy = BanditPolicy(slack_factor = 0.1, evaluation_interval=2)

param_sampling = RandomParameterSampling( {
    '--n_estimator': choice(100, 200, 500, 800, 1000),
    '--min_samples_split': choice(2, 5, 10),
    '--max_features': choice('auto', 'sqrt', 'log2'),
    '--bootstrap': choice(1, 0)
})

if "training" not in os.listdir():
    os.mkdir("./training")

# get registered environment
env = Environment.from_conda_specification(name="sklearn-env",file_path="./conda_dependencies.yaml")

# set up script run configuration
src = ScriptRunConfig(
    source_directory='.',
    script='train.py',
    compute_target=compute_target,
    environment=env
)

hyperdrive_run_config = HyperDriveConfig(run_config = src,
                                     hyperparameter_sampling = param_sampling, 
                                     policy = early_termination_policy,
                                     primary_metric_name = 'Accuracy',
                                     primary_metric_goal = PrimaryMetricGoal.MAXIMIZE,
                                     max_total_runs = 100,
                                     max_concurrent_runs = 4)


In [12]:
# Submit experiment
hyperdrive_run = experiment.submit(hyperdrive_run_config)
RunDetails(hyperdrive_run).show()
hyperdrive_run.wait_for_completion(show_output=True)

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

RunId: HD_a772e5ca-9f91-4150-b600-9fe8c5c4bb9c
Web View: https://ml.azure.com/runs/HD_a772e5ca-9f91-4150-b600-9fe8c5c4bb9c?wsid=/subscriptions/9b72f9e6-56c5-4c16-991b-19c652994860/resourcegroups/aml-quickstarts-251697/workspaces/quick-starts-ws-251697&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254

Streaming azureml-logs/hyperdrive.txt

[2024-02-05T10:58:12.935363][GENERATOR][INFO]Trying to sample '4' jobs from the hyperparameter space
[2024-02-05T10:58:13.5563365Z][SCHEDULER][INFO]Scheduling job, id='HD_a772e5ca-9f91-4150-b600-9fe8c5c4bb9c_0' 
[2024-02-05T10:58:13.6492305Z][SCHEDULER][INFO]Scheduling job, id='HD_a772e5ca-9f91-4150-b600-9fe8c5c4bb9c_1' 
[2024-02-05T10:58:13.778616][GENERATOR][INFO]Successfully sampled '4' jobs, they will soon be submitted to the execution target.
[2024-02-05T10:58:13.8932264Z][SCHEDULER][INFO]Scheduling job, id='HD_a772e5ca-9f91-4150-b600-9fe8c5c4bb9c_3' 
[2024-02-05T10:58:13.9736421Z][SCHEDULER][INFO]Scheduling job, id='HD_a772e5ca-9f91-4150-b600-9fe8c5c4bb

[2024-02-05T11:05:20.193393][GENERATOR][INFO]Trying to sample '1' jobs from the hyperparameter space
[2024-02-05T11:05:20.6116519Z][SCHEDULER][INFO]Scheduling job, id='HD_a772e5ca-9f91-4150-b600-9fe8c5c4bb9c_25' 
[2024-02-05T11:05:20.508009][GENERATOR][INFO]Successfully sampled '1' jobs, they will soon be submitted to the execution target.
[2024-02-05T11:05:20.8919719Z][SCHEDULER][INFO]Successfully scheduled a job. Id='HD_a772e5ca-9f91-4150-b600-9fe8c5c4bb9c_25' 
[2024-02-05T11:05:50.297061][GENERATOR][INFO]Trying to sample '3' jobs from the hyperparameter space
[2024-02-05T11:05:50.6890802Z][SCHEDULER][INFO]Scheduling job, id='HD_a772e5ca-9f91-4150-b600-9fe8c5c4bb9c_26' 
[2024-02-05T11:05:50.8131781Z][SCHEDULER][INFO]Scheduling job, id='HD_a772e5ca-9f91-4150-b600-9fe8c5c4bb9c_27' 
[2024-02-05T11:05:50.9454210Z][SCHEDULER][INFO]Successfully scheduled a job. Id='HD_a772e5ca-9f91-4150-b600-9fe8c5c4bb9c_26' 
[2024-02-05T11:05:50.9736853Z][SCHEDULER][INFO]Scheduling job, id='HD_a772e5ca-9f

[2024-02-05T11:14:51.281922][GENERATOR][INFO]Trying to sample '1' jobs from the hyperparameter space
[2024-02-05T11:14:51.6172447Z][SCHEDULER][INFO]Scheduling job, id='HD_a772e5ca-9f91-4150-b600-9fe8c5c4bb9c_50' 
[2024-02-05T11:14:51.567560][GENERATOR][INFO]Successfully sampled '1' jobs, they will soon be submitted to the execution target.
[2024-02-05T11:14:52.0666593Z][SCHEDULER][INFO]Successfully scheduled a job. Id='HD_a772e5ca-9f91-4150-b600-9fe8c5c4bb9c_50' 
[2024-02-05T11:15:21.284142][GENERATOR][INFO]Trying to sample '3' jobs from the hyperparameter space
[2024-02-05T11:15:21.6185077Z][SCHEDULER][INFO]Scheduling job, id='HD_a772e5ca-9f91-4150-b600-9fe8c5c4bb9c_51' 
[2024-02-05T11:15:21.8400125Z][SCHEDULER][INFO]Scheduling job, id='HD_a772e5ca-9f91-4150-b600-9fe8c5c4bb9c_52' 
[2024-02-05T11:15:21.778391][GENERATOR][INFO]Successfully sampled '3' jobs, they will soon be submitted to the execution target.
[2024-02-05T11:15:21.8516352Z][SCHEDULER][INFO]Successfully scheduled a job. I

[2024-02-05T11:22:21.272953][GENERATOR][INFO]Trying to sample '1' jobs from the hyperparameter space
[2024-02-05T11:22:21.6597548Z][SCHEDULER][INFO]Scheduling job, id='HD_a772e5ca-9f91-4150-b600-9fe8c5c4bb9c_75' 
[2024-02-05T11:22:21.599674][GENERATOR][INFO]Successfully sampled '1' jobs, they will soon be submitted to the execution target.
[2024-02-05T11:22:22.0098786Z][SCHEDULER][INFO]Successfully scheduled a job. Id='HD_a772e5ca-9f91-4150-b600-9fe8c5c4bb9c_75' 
[2024-02-05T11:22:51.317109][GENERATOR][INFO]Trying to sample '2' jobs from the hyperparameter space
[2024-02-05T11:22:51.7633887Z][SCHEDULER][INFO]Scheduling job, id='HD_a772e5ca-9f91-4150-b600-9fe8c5c4bb9c_76' 
[2024-02-05T11:22:51.9316507Z][SCHEDULER][INFO]Scheduling job, id='HD_a772e5ca-9f91-4150-b600-9fe8c5c4bb9c_77' 
[2024-02-05T11:22:51.827518][GENERATOR][INFO]Successfully sampled '2' jobs, they will soon be submitted to the execution target.
[2024-02-05T11:22:52.0376343Z][SCHEDULER][INFO]Successfully scheduled a job. I

{'runId': 'HD_a772e5ca-9f91-4150-b600-9fe8c5c4bb9c',
 'target': 'azurecapstone-project',
 'status': 'Completed',
 'startTimeUtc': '2024-02-05T10:58:12.210326Z',
 'endTimeUtc': '2024-02-05T11:27:31.436926Z',
 'services': {},
 'properties': {'primary_metric_config': '{"name":"Accuracy","goal":"maximize"}',
  'resume_from': 'null',
  'runTemplate': 'HyperDrive',
  'azureml.runsource': 'hyperdrive',
  'platform': 'AML',
  'ContentSnapshotId': '9f037370-52da-4810-9d83-2ea124de7d9e',
  'user_agent': 'python/3.8.5 (Linux-5.15.0-1040-azure-x86_64-with-glibc2.10) msrest/0.7.1 Hyperdrive.Service/1.0.0 Hyperdrive.SDK/core.1.51.0',
  'space_size': '90',
  'best_child_run_id': 'HD_a772e5ca-9f91-4150-b600-9fe8c5c4bb9c_72',
  'score': '0.9766081871345028',
  'best_metric_status': 'Succeeded',
  'best_data_container_id': 'dcid.HD_a772e5ca-9f91-4150-b600-9fe8c5c4bb9c_72'},
 'inputDatasets': [],
 'outputDatasets': [],
 'runDefinition': {'configuration': None,
  'attribution': None,
  'telemetryValues': 

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [29]:
RunDetails(hyperdrive_run).show()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [54]:
best_run = hyperdrive_run.get_best_run_by_primary_metric()
print(best_run.get_details())

{'runId': 'HD_a772e5ca-9f91-4150-b600-9fe8c5c4bb9c_72', 'target': 'azurecapstone-project', 'status': 'Completed', 'startTimeUtc': '2024-02-05T11:21:48.432234Z', 'endTimeUtc': '2024-02-05T11:22:07.022179Z', 'services': {}, 'properties': {'_azureml.ComputeTargetType': 'amlctrain', '_azureml.ClusterName': 'azurecapstone-project', 'ContentSnapshotId': '9f037370-52da-4810-9d83-2ea124de7d9e', 'ProcessInfoFile': 'azureml-logs/process_info.json', 'ProcessStatusFile': 'azureml-logs/process_status.json'}, 'inputDatasets': [], 'outputDatasets': [], 'runDefinition': {'script': 'train.py', 'command': '', 'useAbsolutePath': False, 'arguments': ['--bootstrap', '0', '--max_features', 'auto', '--min_samples_split', '2', '--n_estimator', '100'], 'sourceDirectoryDataStore': None, 'framework': 'Python', 'communicator': 'None', 'target': 'azurecapstone-project', 'dataReferences': {}, 'data': {}, 'outputData': {}, 'datacaches': [], 'jobName': None, 'maxRunDurationSeconds': 2592000, 'nodeCount': 1, 'instance

In [55]:
best_run.get_file_names()

['outputs/.amlignore',
 'outputs/automl-model.pkl',
 'outputs/best_automl_model.onnx',
 'outputs/model.pkl',
 'system_logs/cs_capability/cs-capability.log',
 'system_logs/hosttools_capability/hosttools-capability.log',
 'system_logs/lifecycler/execution-wrapper.log',
 'system_logs/lifecycler/lifecycler.log',
 'system_logs/metrics_capability/metrics-capability.log',
 'system_logs/snapshot_capability/snapshot-capability.log',
 'user_logs/std_log.txt']

In [56]:
# Save the best model
best_run_metrics = best_run.get_metrics()
for metric_name in best_run_metrics:
    metric = best_run_metrics[metric_name]
    print(metric_name, metric)


No of Estimators: 100
No of Features Considered: auto
Min No of Samples to Split: 2
Accuracy 0.9766081871345029
Bootstrap: False


## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.

In [58]:
hypermodel = best_run.register_model(model_name='hdmodel', model_path='outputs/model.pkl') 
#best_run.downlod_file('outputs/model.joblib', './outputs/model.joblib')

In [59]:
from azureml.core.conda_dependencies import CondaDependencies
import sklearn

service_env = Environment('my-sklearn-environment')
service_env_dependencies = CondaDependencies.create(pip_packages=['azureml-defaults', 'inference-schema[numpy-suppot]', 'joblib', 'numpy', 'scikit-learn=={}'.format(sklearn.__version__)])
service_env.python.conda_dependencies = service_env_dependencies

with open('./service_env.yml', "w") as f:
    f.write(service_env_dependencies.serialize_to_string())

In [64]:
import pandas as pd
from azureml.core.webservice import Webservice
from azureml.core.model import InferenceConfig
from azureml.core.environment import Environment
from azureml.core import Workspace
from azureml.core.model import Model
from azureml.automl.core.shared import constants

model = Model(ws, 'hdmodel')

#myenv = automl_best_run.get_environment()
entry_script = './predict.py'
inference_config = InferenceConfig(entry_script=entry_script, environment=service_env)
aci_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=1, description="Breast Cancer Prediction Service")

service = Model.deploy(workspace=ws, 
                       name='hd-breastcancer', 
                       models=[hypermodel], 
                       inference_config=inference_config, 
                       deployment_config=aci_config,
                       overwrite=True)


azureml.core.model:
To leverage new model deployment capabilities, AzureML recommends using CLI/SDK v2 to deploy models as online endpoint, 
please refer to respective documentations 
https://docs.microsoft.com/azure/machine-learning/how-to-deploy-managed-online-endpoints /
https://docs.microsoft.com/azure/machine-learning/how-to-attach-kubernetes-anywhere 
For more information on migration, see https://aka.ms/acimoemigration 
To disable CLI/SDK v1 deprecation warning set AZUREML_LOG_DEPRECATION_WARNING_ENABLED to 'False'


**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.

